In [1]:
from transformers_cocon import (AdamW,GPT2Config,GPT2LMHeadModel,GPT2Tokenizer,
                                         PreTrainedModel,PreTrainedTokenizer,get_linear_schedule_with_warmup,
                                         CoconBlock,HDiscriminator)

In [ ]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../input/yelp-dgr/')

In [2]:
import torch
torch.cuda.is_available()

False

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [4]:
import logging
import math
import os

import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss

import argparse
import glob
import logging
import os
import pickle
import random
import re
import shutil
from typing import Dict, List, Tuple
import json
#import wget
import math

import numpy as np
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from torch.distributed import get_rank, get_world_size
from tqdm import tqdm, trange

from torch.nn import functional as F
from torch.nn.modules.normalization import LayerNorm
from torch.nn.modules import ModuleList
import copy


In [5]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.config.vocab_size

50257

In [6]:
special_tokens = ['<POS>', '<NEG>','<CON_START>','<START>','<END>','<ATTR_WORDS>','<PAD>']
special_tokens_dict = {'additional_special_tokens': special_tokens}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))
start_token_id = tokenizer.convert_tokens_to_ids(['<START>'])[0]
pad_token_id = tokenizer.convert_tokens_to_ids(['<PAD>'])[0]
model.config.vocab_size

50264

In [7]:
for param in model.parameters():
    #print(param)
    param.requires_grad = False

In [20]:
model_state_dict = torch.load('yelp_models/pytorch_model_zero_grad_1.bin', map_location='cpu')
model.load_state_dict(model_state_dict,strict=False) #transformer.h.'0'.attn.masked_bias not loaded as abset in older version of transformers library which is used here
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50263, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (instance_norm): InstanceNorm1d(768, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropou

In [21]:
model.eval()
model.zero_grad()

In [22]:
logger = logging.getLogger(__name__)

In [23]:
config = GPT2Config.from_pretrained("gpt2", cache_dir='saved')

In [24]:
cocon_block = CoconBlock(config.n_ctx, config, scale=True)

In [25]:
cocon_block = cocon_block.to(device)

In [26]:
cocon_block.load_state_dict(torch.load('yelp_models/cocon_dgr_5.pt', map_location='cpu'))

<All keys matched successfully>

In [27]:
model.eval()
model.zero_grad()

cocon_block.eval()
cocon_block.zero_grad()
#cocon_block.train()

# DATASET

In [28]:
import pandas as pd
#df = pd.read_csv('../input/yelp-dgr/clean_text_unigrams_r.csv')
df = pd.read_csv('clean_text_unigrams_r.csv')
df.head()


,sentence,sentiment,c1,a1,r1,del_sent,defr_sent
0,excellent food .,POS,food .,excellent,NaN,<POS><CON_START>food .<START>excellent food .<...,<ATTR_WORDS>excellent<CON_START>food .<START>e...
1,superb customer service .,POS,customer service .,superb,NaN,<POS><CON_START>customer service .<START>super...,<ATTR_WORDS>superb<CON_START>customer service ...
2,they also have daily specials and ice cream wh...,POS,they also have daily specials and ice cream wh...,NaN,NaN,<POS><CON_START>they also have daily specials ...,<ATTR_WORDS>nan<CON_START>they also have daily...
3,it 's a good toasted hoagie .,POS,it 's a good toasted hoagie .,NaN,NaN,<POS><CON_START>it 's a good toasted hoagie .<...,<ATTR_WORDS>nan<CON_START>it 's a good toasted...
4,the staff is friendly .,POS,the staff is .,friendly,hostile,<POS><CON_START>the staff is .<START>the staff...,<ATTR_WORDS>friendly<CON_START>the staff is .<...


In [29]:
len(df)#443261
df_ = df.dropna(subset=['a1'])
len(df_)#254591

254591

In [42]:
df.value_counts(subset=['sentiment'])

sentiment
POS          266042
NEG          177219
dtype: int64

In [43]:
df_.value_counts(subset=['sentiment'])

sentiment
POS          175768
NEG           78823
dtype: int64

In [ ]:
tokenizer.decode(train_ds[0]), tokenizer.decode(eval_ds[0])

In [30]:
from sklearn.model_selection import train_test_split
inputs = df_['del_sent'].to_list()
y = df_['a1'].to_list()
train_ds, eval_ds, train_attr, eval_attr = train_test_split(inputs, y, test_size=0.33, random_state=42)
    

In [ ]:
len(train_ds)+len(eval_ds), len(df_),len(train_attr), len(eval_attr)

In [31]:
def tokenize_and_encode(lines):
        '''
        This method tokenizes the input data and encodes it using the OpenAIGPTTokenizer
        :param file_path: Path of the input file, dtype: str
        :return: encoded dataset  dtype: list
        '''
        tokenized_dataset = lines
        for i, line in enumerate(tqdm(lines)):
            token = tokenizer.tokenize(line)[:512]
            tokenized_dataset[i] = tokenizer.convert_tokens_to_ids(token)
        return tokenized_dataset

In [32]:
train_dataset = tokenize_and_encode(train_ds)
eval_dataset = tokenize_and_encode(eval_ds)
train_attr = tokenize_and_encode(train_attr)
eval_attr = tokenize_and_encode(eval_attr)
input_length = max(max(len(t) for t in train_dataset), max(len(q) for q in eval_dataset))
print('input length ', input_length)
input_length = min(input_length, 85)
print('input length ', input_length)

100%|█████████████████████████████████████████████████████████████████████████| 84016/84016 [00:03<00:00, 22196.43it/s]

input length  51
input length  51


In [33]:
def pre_process_dataset(encoded_dataset, encoded_attr, input_length, start_token_id,pad_token_id=pad_token_id):
        """
        This method is to create torch tensor of input ids and lm labels
        :param encoded_dataset: Input dataset, dtype: list
        :param input_length: Maximum length of sentence from training and eval dataset, dtype: int
        :param start_token_id: id of the '<START>' token, dtype: int
        :return: torch.tensor of size [len(encoded_dataset), 2]
        """

        n_batch = len(encoded_dataset)
        attr_len = max([len(i) for i in encoded_attr])
        attr = np.full(shape=(n_batch, attr_len), fill_value=pad_token_id,  dtype=np.int64)
        input_ids = np.full(shape=(n_batch, input_length), fill_value=pad_token_id,  dtype=np.int64)
        lm_labels = np.full(shape=(n_batch, input_length), fill_value=-100, dtype=np.int64)

        for i, tokens in enumerate(encoded_dataset):
            try:
                #tokens = tokens[:input_length]
                start_id_index = tokens.index(start_token_id)
                input_ids[i, :len(tokens)] = tokens
                start_id_index = tokens.index(start_token_id)
                lm_labels[i, start_id_index : len(tokens)-1] = tokens[start_id_index + 1: len(tokens)]
                attr[i, :len(encoded_attr[i])] = encoded_attr[i]
                # LM loss calculate only for tokens after <START> token in the sentence
                #lm_labels[i, :len(tokens)-1] = tokens[1:]
            except ValueError:
                print("Index {} doesn't have start token".format(i))

        input_ids = torch.tensor(input_ids)
        lm_labels = torch.tensor(lm_labels)
        attr = torch.tensor(attr)
        tensor_dataset = (input_ids, attr, lm_labels)
        #tensor_dataset.append(torch.tensor(d) for d in all_inputs)

        return tensor_dataset

In [34]:
train_batch_size = 32
num_train_epochs = 2
learning_rate = 6.25e-5
warmup_proportion = 0.002
max_grad_norm = 1
weight_decay = 0.01
n_gpu = 1

In [35]:
from torch.utils.data import TensorDataset

In [36]:
train_tensor_dataset = pre_process_dataset(train_dataset,train_attr, input_length, start_token_id=start_token_id)
eval_tensor_dataset = pre_process_dataset(eval_dataset, eval_attr, input_length, start_token_id=start_token_id)

print("Training Example Input ids= {}".format(train_tensor_dataset[0][0]))
print("Training Example attribute ids = {}".format(train_tensor_dataset[1][0]))
print("Training Example Language Modeling ids = {}".format(train_tensor_dataset[2][0]))


train_data = TensorDataset(*train_tensor_dataset)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=train_batch_size)

eval_data = TensorDataset(*eval_tensor_dataset)
eval_sampler = RandomSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=train_batch_size)


Training Example Input ids= tensor([50258, 50259,    72,   326,  1312,   373, 24053,   764, 50260,    72,
         5100,   326,  1312,   373, 24053,   764, 50261, 50262, 50262, 50262,
        50262, 50262, 50262, 50262, 50262, 50262, 50262, 50262, 50262, 50262,
        50262, 50262, 50262, 50262, 50262, 50262, 50262, 50262, 50262, 50262,
        50262, 50262, 50262, 50262, 50262, 50262, 50262, 50262, 50262, 50262,
        50262])
Training Example attribute ids = tensor([45956,   515, 50262, 50262, 50262, 50262, 50262, 50262, 50262, 50262])
Training Example Language Modeling ids = tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,    72,  5100,
          326,  1312,   373, 24053,   764, 50261,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100])


In [ ]:
#Commented part corresponds to Cycle loss code

def train_cocon(train_dataset=train_dataloader, model=model, tokenizer=tokenizer, cocon_block=cocon_block):
    t_total = len(train_dataloader) * num_train_epochs
    
    learning_rate = 5e-5
    adam_epsilon = 1e-8
    #t_total = len(train_dataloader) #// args.gradient_accumulation_steps * args.num_train_epochs

    no_decay = ["bias", "LayerNorm.weight"]
    cocon_block_optimizer_grouped_parameters = [
        {
            "params": [p for n, p in cocon_block.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
        {"params": [p for n, p in cocon_block.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
        ]
    cocon_block_optimizer = AdamW(cocon_block_optimizer_grouped_parameters, lr=learning_rate, eps=adam_epsilon)
    cocon_block_scheduler = get_linear_schedule_with_warmup(
        cocon_block_optimizer, num_warmup_steps=0, num_training_steps=t_total)
    
    #model, cocon_block, cocon_block_optimizer, train_dataloader = accelerator.prepare(model, cocon_block, cocon_block_optimizer, train_dataloader)
    #cocon_block_optimizer.to(device)
    #train_dataloader.to(device)
    
    
    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    tr_loss, logging_loss = 0.0, 0.0
    #total_loss = 0
    lambda_self = 1
    lambda_null = 1
    lambda_cycle = 1
    #epoch_max_steps = 10
    #max_steps = 100
    #hs_len = 0 
    #cs_len = 20 
    #tis_len = 20 
    
    #model.zero_grad()
    train_iterator = trange(epochs_trained, num_train_epochs, desc="Epoch")
    for epoch_ind in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration")
        for step, batch in enumerate(epoch_iterator):
            
            inputs, content, lm_labels = batch
            #print(inputs.shape, content.shape, lm_labels.shape)
            
            #if(inputs.shape[1]<hs_len):
            #    continue
            
            inputs = inputs.to(device)
            lm_labels = lm_labels.to(device)
            content = content.to(device)

            #hs_tis_split_ind = random.randint(0,2)
            #hs_len = 10 + hs_tis_split_ind
            #cs_len = 20 - hs_tis_split_ind
            #tis_len = 20 - hs_tis_split_ind
            
            #lm_labels = lm_labels[:, :hs_len+tis_len]
            #original_context_seq = inputs[:, hs_len:hs_len+cs_len]
            #original_history_seq = inputs[:, :hs_len]
            #original_transform_input_seq = inputs[:, hs_len:hs_len+tis_len]
            
            #other_sample_inputs = torch.cat([inputs[-1:], inputs[:-1]], dim=0)
            #other_sample_lm_labels = other_sample_inputs[:, :hs_len+tis_len]

            #other_sample_context_seq = other_sample_inputs[:, hs_len:hs_len+cs_len]
            #other_sample_history_seq = other_sample_inputs[:, :hs_len]
            #other_sample_transform_input_seq = other_sample_inputs[:, hs_len:hs_len+tis_len]
            
            #model.eval()
            #cocon_block.train()
            model.zero_grad()
            cocon_block.zero_grad()
            
            #SELF LOSS -> Modified to sentiment loss
            #forward pass L_alpha(entire_input) #NO AR
            #forward pass L_alpha(original_content)
            with torch.no_grad():
                hidden_states = model(inputs, output_after_block_ind=5)[0] 
                context_seq_hidden_states =  model(content, output_after_block_ind=5)[0] 
           
            original_hidden_states = hidden_states.to(device)
            #print(original_hidden_states.shape)
            #original_history_seq_hidden_states = original_hidden_states[:, :hs_len].to(device)
            #original_transform_input_seq_hidden_states = original_hidden_states[:, hs_len:hs_len+tis_len].to(device)
            original_context_seq_hidden_states = context_seq_hidden_states.to(device)
            #print(original_context_seq_hidden_states.shape)
    
            #other_sample_hidden_states = torch.cat([hidden_states[-1:], hidden_states[:-1]], dim=0).to(device)
            #other_sample_history_seq_hidden_states = other_sample_hidden_states[:, :hs_len].to(device)
            #other_sample_transform_input_seq_hidden_states = other_sample_hidden_states[:, hs_len:hs_len+tis_len].to(device)
    
            #other_sample_context_seq_hidden_states = torch.cat([context_seq_hidden_states[-1:], context_seq_hidden_states[:-1]], dim=0).to(device)
            
            #Cocon(input_hidden, content_hidden, history_hidden) #ROLE of history in cocon = concats to input
            self_cocon_hidden_states = cocon_block(original_hidden_states, 
                                       context_seq=original_context_seq_hidden_states, 
                                       include_sos_output=True, cs_self_attn_mask_prob=1)
            
            #print('C: ',self_cocon_hidden_states.shape)
            
            #self_cocon_hidden_states = cocon_block(original_context_seq_hidden_states, 
                                       #include_sos_output=True, cs_self_attn_mask_prob=0)
            
            #print('C: ',self_cocon_hidden_states.shape)

            
            #self_cocon_lm_tail_input = torch.cat([original_hidden_states[:, :-1], self_cocon_hidden_states], dim=1).to(device)
            #print('MUST MATCH LM LABELS: ',self_cocon_hidden_states.shape)
            #print(lm_labels.shape)

            # Ignore history when computing loss
            #lm_logit_first_index = original_history_seq_hidden_states.shape[1] -1
            #lm_labels_first_index = lm_logit_first_index + 1
            
            #L_beta([original_history_hidden + cocon_hidden])
            self_cocon_lm_tail_outputs = model(input_hidden_state=self_cocon_hidden_states, 
                                               labels=lm_labels, lm_logit_first_index=0, 
                                               lm_labels_first_index=0, 
                                               input_before_block_ind=5+1)
                        
            #self_cocon_lm_tail_outputs[0][1] == self_cocon_lm_tail_outputs[1]
            #next_token_logits = self_cocon_lm_tail_outputs[1]  # [N,L,C] where C is vocab_size
            #if next_token_logits.shape[1] > 1:
            #    next_token_logits = next_token_logits[:, -1:]
            #next_cocon_output_prob = torch.nn.functional.softmax(next_token_logits, dim=-1)
            
            self_cocon_lm_loss = self_cocon_lm_tail_outputs[0] #SELF LOSS
            total_loss = lambda_self * self_cocon_lm_loss
            
            #NULL LOSS ->modified to target sentiment
            #L_alpha for input already done above
            #L_alpha for content not req as null
            #Cocon block [input_hidden, history]
            #with torch.no_grad():
            #    context_seq_hidden_states_null = model(target, path='half1')
            #    context_seq_hidden_states_null = context_seq_hidden_states_null.to(device)

            null_cocon_hidden_states = cocon_block(original_hidden_states, 
                                       context_seq=None,
                                       include_sos_output=True)#, cs_self_attn_mask_prob=1- not req as noncontet to copy from
            
            #null_cocon_lm_tail_input = null_cocon_hidden_states

            # Ignore history when computing loss - same as self
            #lm_logit_first_index = original_history_seq_hidden_states.shape[1] -1
            #lm_labels_first_index = lm_logit_first_index + 1
            
            #L_beta([original_history_hidden + cocon_hidden])
            null_cocon_lm_tail_outputs = model(input_hidden_state=null_cocon_hidden_states, labels=lm_labels, 
                                               lm_logit_first_index=0, 
                                               lm_labels_first_index=0, 
                                               input_before_block_ind=5+1)
            #self_cocon_lm_tail_outputs[0][1] == self_cocon_lm_tail_outputs[1]
            #next_token_logits = self_cocon_lm_tail_outputs[1]  # [N,L,C] where C is vocab_size
            #if next_token_logits.shape[1] > 1:
            #    next_token_logits = next_token_logits[:, -1:]
            #next_cocon_output_prob = torch.nn.functional.softmax(next_token_logits, dim=-1)
            
            null_cocon_lm_loss = null_cocon_lm_tail_outputs[0] #SELF LOSS
            total_loss += lambda_null * null_cocon_lm_loss

            cur_len = 0

            """
            #CYCLE LOSS
            #step1 : [other history, original content] -> compute mixed output
            #Done AR so start with input=None
             
            cocon_block_output = None
            cocon_th_gen_output = None 
            cocon_th_gen_input = None #[same as output but detatched]
            cocon_output_embed = None #[used as content for step 2, instead of converting to token and back to hidden]

            lm_tail_past = False #To indicate if consider entire embeding or only last one
            lm_head_past = False #same as above
            max_ar_len = original_transform_input_seq_hidden_states.shape[1]#20
            
            other_sample_history_seq_one_hot_prob = to_one_hot(other_sample_history_seq, n_dims=config.vocab_size).to(device)
            other_sample_history_seq_embeds = torch.matmul(other_sample_history_seq_one_hot_prob, model.wte.weight).to(device)
            
            #Start auto regressive generation
            while cur_len < max_ar_len:
                #skipping L_alpha as we already have hidden states from previous computations
                cocon_transformed_hidden_states = cocon_block(cocon_th_gen_input, context_seq=original_context_seq_hidden_states, history_seq=other_sample_history_seq_hidden_states, include_sos_output=True, cs_self_attn_mask_prob=1)
                if cur_len == 0:
                    cocon_block_output = cocon_transformed_hidden_states[:, -1:] #Consider last token as output
                else:
                    cocon_block_output = torch.cat([cocon_block_output, cocon_transformed_hidden_states[:,-1:]], dim=1)
                
                #
                if cocon_th_gen_input is not None:
                    hist_plus_cocon_hidden_states = torch.cat([other_sample_history_seq_hidden_states, cocon_th_gen_input[:, :-1], cocon_transformed_hidden_states[:, -1:]], dim=1)
                else:
                    hist_plus_cocon_hidden_states = torch.cat([other_sample_history_seq_hidden_states[:, :-1], cocon_transformed_hidden_states[:, -1:]], dim=1)
                    
                if(lm_tail_past):
                    lm_tail_inputs = hist_plus_cocon_hidden_states[:, -1:, :].to(device)
                else:
                    lm_tail_inputs = hist_plus_cocon_hidden_states.to(device)
                    
                tail_outputs = model(lm_tail_inputs,path='half2')
                next_token_logits = tail_outputs[1]  # [N,L,C] where C is vocab_size
                if next_token_logits.shape[1] > 1:
                    next_token_logits = next_token_logits[:, -1:]
                lm_tail_past = True
                next_cocon_output_prob = torch.nn.functional.softmax(next_token_logits, dim=-1)
                next_cocon_output_embed = torch.matmul(next_cocon_output_prob, model.wte.weight).to(device)
                if cur_len==0:
                    cocon_output_embeds = next_cocon_output_embed.to(device)
                    hist_plus_cocon_output_embeds = torch.cat([other_sample_history_seq_embeds, next_cocon_output_embed], dim=1).to(device)
                else:
                    cocon_output_embeds = torch.cat([cocon_output_embeds, next_cocon_output_embed], dim=1).to(device)
                    hist_plus_cocon_output_embeds = torch.cat([hist_plus_cocon_output_embeds, next_cocon_output_embed], dim=1).to(device)
                    
                if(lm_head_past):
                    lm_head_inputs = hist_plus_cocon_output_embeds[:, -1:, :].to(device)
                else:
                    lm_head_inputs = hist_plus_cocon_output_embeds.to(device)
                    
                head_outputs = model(lm_head_inputs, inputs_embeds=True)
                cocon_gen_output_h = head_outputs[0][1] #tail_outputs[0][1].shape
                if cocon_gen_output_h.shape[1] > 1:
                    next_h = cocon_gen_output_h[:, -1:]
                else:
                    next_h = cocon_gen_output_h
                lm_head_past = True
                    
                    
                h_to_cat_input = next_h.detach()
                if(cur_len ==0):
                    cocon_th_gen_input = h_to_cat_input.to(device)
                    cocon_th_gen_output = next_h.to(device)
                else:
                    cocon_th_gen_input = torch.cat([cocon_th_gen_input, h_to_cat_input], dim=1).to(device)
                    cocon_th_gen_output = torch.cat([cocon_th_gen_output, next_h], dim=1).to(device)
                
                cur_len = cocon_th_gen_input.shape[1]
            
            #Completed AR generation 
            ar_cocon_final_output_embeds = cocon_output_embeds
            
            #STEP 1 - other history, original content = mixed
            other_context_cocon_hidden_states = cocon_block(cocon_th_gen_output, context_seq=original_context_seq_hidden_states, history_seq=other_sample_history_seq_hidden_states, include_sos_output=True,cs_self_attn_mask_prob=1)
            other_context_cocon_lm_tail_input = torch.cat([other_sample_history_seq_hidden_states[:, :-1], other_context_cocon_hidden_states], dim=1)
            lm_logit_first_index = other_sample_history_seq_hidden_states.shape[1] -1
            lm_labels_first_index = lm_logit_first_index + 1
            
            other_context_cocon_lm_tail_outputs = model(other_context_cocon_lm_tail_input, labels=other_sample_lm_labels, lm_logit_first_index=lm_logit_first_index, lm_labels_first_index=lm_labels_first_index, path='half2')
            other_contex_cocon_lm_loss = other_context_cocon_lm_tail_outputs[0][0]
            total_loss += other_contex_cocon_lm_loss * lambda_cycle
            
            
            #STEP2 - original history, mixed content = original content
            #to get mixed content we use the embedding directly to get hidden state instead of starting from tokens
            ar_cocon_output_hidden_states = model(ar_cocon_final_output_embeds, inputs_embeds=True, path='half1')
            #cocon - [original_transformed, mixed_content, original_history]
            cycle_ar_cocon_recon_hidden_states = cocon_block(original_transform_input_seq_hidden_states, context_seq=ar_cocon_output_hidden_states, history_seq=original_history_seq_hidden_states, include_sos_output=True, cs_self_attn_mask_prob=1)
            #update history to include cocon output
            cycle_ar_cocon_recon_lm_tail_input = torch.cat([original_history_seq_hidden_states[:, :-1], cycle_ar_cocon_recon_hidden_states], dim=1)
            #index after history
            lm_logit_first_index = original_history_seq_hidden_states.shape[1] -1
            #L_beta prediction
            cycle_ar_cocon_recon_lm_tail_outputs = model(cycle_ar_cocon_recon_lm_tail_input, labels=lm_labels, lm_logit_first_index=lm_logit_first_index, lm_labels_first_index=lm_labels_first_index, path='half2')
            
            cycle_ar_cocon_recon_lm_loss = cycle_ar_cocon_recon_lm_tail_outputs[0][0]
            total_loss += cycle_ar_cocon_recon_lm_loss * lambda_cycle
            """
            
            total_loss.backward()
            #accelerator.backward(total_loss)
        
            tr_loss += total_loss.item()

            #eval_loss, perplexity = evaluate()
        
            if (step + 1) % 100 == 0: 
                #ADD validation , bleue TODO
                print("loss: ", tr_loss/(step+1))
                torch.nn.utils.clip_grad_norm_(cocon_block.parameters(), 1)
                cocon_block_optimizer.step() # opt.step() does not zero_grad, need to zero.grad() manually
                cocon_block_scheduler.step() # Update learning rate schedule 
                torch.save({
                          'epoch': epoch_ind,
                          'model_state_dict': cocon_block.state_dict(),
                          'optimizer_state_dict': cocon_block_optimizer.state_dict(),
                          'loss': tr_loss,
                          'step': step}, 'modified.tar')
                
                cocon_block.zero_grad()
                model.zero_grad()
            
            global_step +=1
            #if(step> epoch_max_steps or global_step>max_steps):
             #   epoch_iterator.close()
             #   break
        #if(global_step>max_steps):
        #    train_iterator.close()
        #    break
    return global_step, tr_loss/global_step

In [53]:
global_step, tr_loss = train_cocon()

In [54]:
global_step, tr_loss #2 epochs = (10662, 20.512922179115307)
#4 epochs (10662, 10.538968369518473)
# 6 epochs (10662, 9.399290811856767)
#8 epochs (10662, 8.746620702734585)
#10 epochs (10662, 8.746620702734585)

In [55]:
torch.save(cocon_block.state_dict(),'cocon_dgr_5.pt')

In [ ]:
model = TheModelClass(*args, **kwargs)
optimizer = TheOptimizerClass(*args, **kwargs)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

# decoding

In [6]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

model_state_dict = torch.load('yelp_models/pytorch_model_zero_grad_1.bin',map_location=torch.device('cpu'))

special_tokens = ['<POS>', '<NEG>','<CON_START>','<START>','<END>','<PAD>']
special_tokens_dict = {'additional_special_tokens': special_tokens}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
start_token_id = tokenizer.convert_tokens_to_ids(['<START>'])[0]
pad_token_id = tokenizer.convert_tokens_to_ids(['<PAD>'])[0]

model.resize_token_embeddings(len(tokenizer))

model.load_state_dict(model_state_dict, strict=False)
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50263, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (instance_norm): InstanceNorm1d(768, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropou

In [40]:
tokenizer.encode('<START> sun shines in the sky <END> <PAD> <PAD>')

[50260, 4252, 32481, 287, 262, 6766, 50261, 50262, 50262]

In [7]:
cocon_block = CoconBlock(model.config.n_ctx, model.config, scale=True)
cocon_block = cocon_block.to(device)

In [8]:
cocon_block.load_state_dict(torch.load('yelp_models/cocon_dg_3l_pretained_gpt_2.pt', map_location=torch.device('cpu')))

<All keys matched successfully>

In [9]:
end_token_id = tokenizer.convert_tokens_to_ids(['<END>'])[0]
pad_token_id = tokenizer.convert_tokens_to_ids(['<PAD>'])[0]

In [37]:
end_token_id, pad_token_id

(50261, 50262)

In [11]:
def preditction_with_beam_search(ref_text, content=None,beam_width=3, max_len=30,vocab_length=40483, end_token_id=end_token_id ):
    """
    This function decodes sentences using Beam Seach. 
    It will output #sentences = beam_width. This function works on a single example.
    
    ref_text : string : Input sentence
    beam_width : int : Width of the output beam
    vocab_length : int : Size of the Vocab after adding the special tokens
    """
    
    done = [False for i in range(beam_width)] # To track which beams are already decoded
    stop_decode = False
    decoded_sentences=[] # List of decoded sentences at any given time
    
    sm = torch.nn.Softmax(dim=-1) # To calculate Softmax over the final layer Logits
    tokens = tokenizer.tokenize(ref_text) # Tokenize the input text
    if(content is not None):
        content_tokens =  tokenizer.tokenize(content)
        content_indexed_tokens = tokenizer.convert_tokens_to_ids(content_tokens)
        content_index_tokens = [content_indexed_tokens for i in range(beam_width)]
        content_torch_tensor = torch.tensor(content_index_tokens).to(device)
    
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokens) # Convert tokens to ids
    index_tokens = [indexed_tokens for i in range(beam_width)] # Replication of Input ids for all the beams

    #index_tokens = [indexed_tokens for i in range(beam_width)]
    torch_tensor = torch.tensor(index_tokens).to(device)
    beam_indexes = [[] for i in range(beam_width)] # indexes of the current decoded beams
    best_scoes = [0 for i in range(beam_width)] # A list of lists to store Probability values of each decoded token of best beams
    count = 0
    while count < max_len and not stop_decode:
        if count == 0: # For the first step when only one sentence is availabe
            with torch.no_grad():
                # Calculate output probability distribution over the Vocab,
                hidden_states = model(torch_tensor, output_after_block_ind=5)[0] 
                if(content is not None):
                    context_seq_hidden_states =  model(content_torch_tensor, output_after_block_ind=5)[0] 
                else:
                    context_seq_hidden_states = None
           
                original_hidden_states = hidden_states.to(device)
                original_context_seq_hidden_states = context_seq_hidden_states.to(device)
                
                self_cocon_hidden_states = cocon_block(original_hidden_states, 
                                       context_seq=original_context_seq_hidden_states, 
                                       include_sos_output=True, cs_self_attn_mask_prob=0)
        
                self_cocon_lm_tail_outputs = model(input_hidden_state=self_cocon_hidden_states,lm_logit_first_index=0, 
                                               lm_labels_first_index=0, 
                                               input_before_block_ind=5+1)
            
            #print(self_cocon_lm_tail_outputs[0].shape,self_cocon_lm_tail_outputs[1].shape)
                #output = model(torch_tensor)
                preds = sm(self_cocon_lm_tail_outputs[0]) #  shape = [beam_bidth, len(input_sen)+1,Vocab_length]
            top_v, top_i = preds[:,-1,:].topk(beam_width) # Fatch top indexes and it's values
            [beam_indexes[i].append(top_i[0][i].tolist()) for i in range(beam_width)] # Update the Beam indexes
            # Update the best_scores, for first time just add the topk values directly
            for i in range(beam_width):
                best_scoes[i] = top_v[0][i].item()
            count += 1
        else: # After first step
            # Prepare the current_state by concating original input and decoded beam indexes
            current_state = torch.cat((torch_tensor, torch.tensor(beam_indexes).to(device)), dim=1)
            # Prediction on the current state
            with torch.no_grad():
                hidden_states = model(current_state, output_after_block_ind=5)[0] 
                #if(content is not None):
                 #   context_seq_hidden_states =  model(content_torch_tensor, output_after_block_ind=5)[0] 
                #else:
                #    context_seq_hidden_states = None
           
                original_hidden_states = hidden_states.to(device)
                #original_context_seq_hidden_states = context_seq_hidden_states.to(device)
                
                self_cocon_hidden_states = cocon_block(original_hidden_states, 
                                       context_seq=original_context_seq_hidden_states, 
                                       include_sos_output=True, cs_self_attn_mask_prob=0)
        
                self_cocon_lm_tail_outputs = model(input_hidden_state=self_cocon_hidden_states,lm_logit_first_index=0, 
                                               lm_labels_first_index=0, 
                                               input_before_block_ind=5+1)
                
                
                #outputs = model(current_state)
                preds = sm(self_cocon_lm_tail_outputs[0])
            # Multiply new probability predictions with corresponding best scores
            # Total socres = beam_width * Vocab_Size
            flatten_score = (preds[:,-1,:]*torch.tensor(best_scoes).to(device).unsqueeze(1)).view(-1)
            # Fatch the top scores and indexes 
            vals, inx = flatten_score.topk(beam_width)
            # best_score_inx saves the index of best beams after multiplying the probability of new prediction
            best_scoes_inx = (inx / vocab_length).tolist()
            best_scoes = vals.tolist()
            # Unflatten the index 
            correct_inx = (inx % vocab_length).tolist()
            
            # Check if done for all the Beams
            for i in range(beam_width):
                if correct_inx[i] == end_token_id:
                    done[i] = True
            # Update the best score for each the current Beams
            for i in range(beam_width):
                if not done[i]:
                    best_scoes[i] = vals.tolist()[i]
            # Check is All the Beams are Done
            if (sum(done) == beam_width):
                stop_decode = True
            # Prepapre the new beams
            temp_lt=[0 for i in range(beam_width)]
            for i,x in enumerate(best_scoes_inx):
                temp_lt[i] = beam_indexes[i] + [correct_inx[i]]
            # Update the Beam indexes
            beam_indexes = temp_lt
            del temp_lt
            count += 1
    # Decode All the beam indexes to till <END> token only and convert into sentence
    for i in range(beam_width):
        try:
            end_index = beam_indexes[i].index(end_token_id )
        except ValueError:
            end_index = len(beam_indexes[i])
            
        decoded_sentences.append(tokenizer.decode(beam_indexes[i][:end_index]))
        
    return decoded_sentences

In [12]:
def generate_gpt(inp,cont,gen_len=30):
    input_token = torch.tensor(tokenizer.encode(inp))
    content_token = torch.tensor(tokenizer.encode(cont))
    if(len(input_token.shape)<3):
        input_token = input_token.unsqueeze(0) #batch dim
        content_token = content_token.unsqueeze(0) #batch dim

    #Repeat for history TO DO
    #implement auto regression TODO
    input_token = input_token.to(device)
    content_token = content_token.to(device)
    l = len(input_token[0])
    for i in range(gen_len):
        #L_alpha
        with torch.no_grad():
            hidden_inp = model(input_token, output_after_block_ind=5)[0] 
            hidden_content = model(content_token, output_after_block_ind=5)[0] 
            #Cocon             other_context_cocon_hidden_states = cocon_block(cocon_th_gen_output, context_seq=original_context_seq_hidden_states, history_seq=other_sample_history_seq_hidden_states, include_sos_output=True,cs_self_attn_mask_prob=1)
            cout = cocon_block(hidden_inp, context_seq=hidden_content)
            output = model(input_hidden_state=cout,lm_logit_first_index=0, 
                                               lm_labels_first_index=0, 
                                               input_before_block_ind=5+1)
            pred_token_logits = output[0][:,-1:] 
        #softmax
        pred_token_prob = torch.nn.functional.softmax(pred_token_logits, dim=-1)#[:,-1,:]
        #sample
        pred_token = torch.multinomial(pred_token_prob[0], num_samples=1) #repeat for every elem in batch
        #append
        input_token = torch.cat((input_token,pred_token),1)
        #decode
    #pred_text = tokenizer.decode(input_token)
    return input_token, [tokenizer.decode(i) for i in input_token[:,l:]]

In [13]:
import pandas as pd
df_gpt2_cocon = pd.read_csv('yelp_models/cocon_gpt_preds.csv')
df_gpt2_cocon.head()

,input,gold,content,pred1,pred2,pred_mp,plain_pred1,plain_pred2,plain_pred_mp
0,<POS> <CON_START> i recommend checking this pl...,i highly recommend checking this place out.,highly,.always actors fog fog. fog. fog.,always proposal great actors. fogalways actor...,small definitely. fantastic.always.they recom...,highly highly checking place. highly highly t...,recommenditageitageitageitageitageitageitagei...,highly highly checking place. highly highly t...
1,"<NEG> <CON_START> not only is there pizza, but...","not only is there pizza bad, but their custome...",bad horrible,.ues.always actors fog. actors. actors,alwaysalways actorsgreat. great actorsalways ...,.again.wrong always too rude always.att,only is pizza but customer is. Kurd not is,"even there qualified, wantingetooth wanting w...","only is pizza, their service horrible horribl..."
2,<POS> <CON_START> cool tram that has views goi...,cool tram that has great views going up or dow...,great,always actors actors actors actors actors Kur...,love great always always fog always actors Ku...,owned awesome.keep no months.cious best love,tram has views up down down theitt extended e...,cool always great Mayor MayorGood Mayor pesti...,ixed tram has going or of psburgh skyline <END>
3,<POS> <CON_START> she was! <START>,she was fantastic!,fantastic,always!always! adulthood always actors adulth...,love. travel.always removing. travel..,spot! delicious always great <END> excellent...,was!she amazing Kurd!she amazing Kurd Kurd,she! Kurd scarthink Quebecthink awesome fogthink,was!she phenomenal <END> was! <END> was!
4,<NEG> <CON_START> however the food - oh the fo...,however the food - oh the food : ( - i was dis...,disappointed,always. actors actors great. actors actors.al...,. Portlandalways.. Portlandalways.! always,unfortunately.best.worst.always. awful.,ever food oh food ( - i disappointed disappoin...,ever thearia the bad i - scar forecast disapp...,ever food oh food food disgusting ( i sgy


In [44]:
input_ids = list(df_gpt2_cocon['input'])
content_ids = list(df_gpt2_cocon['content'])


In [43]:
encoded_inputs = 
input_ids[0], tokenizer.encode(input_ids[0])

('<POS> <CON_START> i recommend checking this place out. <START>',
 [50257, 50259, 1312, 4313, 10627, 428, 1295, 503, 13, 50260])

In [45]:
input_ids[0]

'<POS> <CON_START> i recommend checking this place out. <START>'

In [29]:
input_ids =torch.tensor(tokenizer.encode(input_ids))
content_ids = torch.tensor(tokenizer.encode(content_ids))

In [32]:
input_ids[3], content_ids[0]

(tensor(50256), tensor(47444))

In [24]:
input_token = torch.tensor(tokenizer.encode(input_ids[0]))
content_token = torch.tensor(tokenizer.encode(content_ids[0]))
if(len(input_token.shape)<3):
    input_token = input_token.unsqueeze(0) #batch dim
    content_token = content_token.unsqueeze(0)
    
input_token, content_token

(tensor([[50257, 50259,  1312,  4313, 10627,   428,  1295,   503,    13, 50260]]),
 tensor([[47444]]))

In [48]:

outs_mp = []
outs1 = []
outs2 = []
#tqdm_bar = tqdm(eval_dataloader, desc="batch iteration")
for i in range(len(input_ids)):
    op=preditction_with_beam_search(input_ids[i],content_ids[i],2, 10)
    outs1.append(op[0])
    outs2.append(op[1])
    op = generate_gpt(input_ids[i],content_ids[i],gen_len=10)
    outs_mp.append(op[1][0])
    #print(input_ids[0], content_ids[0])
    #break


df_gpt2_cocon['plain_r_pred1'] = outs1
df_gpt2_cocon['plain_r_pred2'] = outs2
df_gpt2_cocon['plain_r_pred_mp'] = outs_mp
#print(outs1, outs2, outs_mp)
    #df_gpt2.to_csv('yelp_model/plain_gpt.csv', index=False)
df_gpt2_cocon.head()

,input,gold,content,pred1,pred2,pred_mp,plain_pred1,plain_pred2,plain_pred_mp,both_3l_pred1,both_3l_pred2,both_3l_pred_mp
0,<POS> <CON_START> i recommend checking this pl...,i highly recommend checking this place out.,highly,.always actors fog fog. fog. fog.,always proposal great actors. fogalways actor...,small definitely. fantastic.always.they recom...,highly highly checking place. highly highly t...,recommenditageitageitageitageitageitageitagei...,highly highly checking place. highly highly t...,recommend this out.itage actorsigaitage love ...,definitely brushelled highly. highly recommen...,recommend this out you. main mindiquette love...
1,"<NEG> <CON_START> not only is there pizza, but...","not only is there pizza bad, but their custome...",bad horrible,.ues.always actors fog. actors. actors,alwaysalways actorsgreat. great actorsalways ...,.again.wrong always too rude always.att,only is pizza but customer is. Kurd not is,"even there qualified, wantingetooth wanting w...","only is pizza, their service horrible horribl...",also there'); is pizza but customer is..,only is pizza mit decrease collective decreas...,also there is pizza but customer isable <END>...
2,<POS> <CON_START> cool tram that has views goi...,cool tram that has great views going up or dow...,great,always actors actors actors actors actors Kur...,love great always always fog always actors Ku...,owned awesome.keep no months.cious best love,tram has views up down down theitt extended e...,cool always great Mayor MayorGood Mayor pesti...,ixed tram has going or of psburgh skyline <END>,great tram has views up down down the of p,cool Mayor Mayor MayorGood or hotel down pestial,cool that views going or of psburgh skyline the
3,<POS> <CON_START> she was! <START>,she was fantastic!,fantastic,always!always! adulthood always actors adulth...,love. travel.always removing. travel..,spot! delicious always great <END> excellent...,was!she amazing Kurd!she amazing Kurd Kurd,she! Kurd scarthink Quebecthink awesome fogthink,was!she phenomenal <END> was! <END> was!,was!she was! gene! Kurd was!,she. she scar. Kurd actors gene!doors,you was!.. <PAD> <PAD> <PAD> <PAD> <PAD>
4,<NEG> <CON_START> however the food - oh the fo...,however the food - oh the food : ( - i was dis...,disappointed,always. actors actors great. actors actors.al...,. Portlandalways.. Portlandalways.! always,unfortunately.best.worst.always. awful.,ever food oh food ( - i disappointed disappoin...,ever thearia the bad i - scar forecast disapp...,ever food oh food food disgusting ( i sgy,the - the : - food i i WAR.,love food Montgomery Montgomery i tempor temp...,them- food oh nasty food oh terrible!!


In [47]:
df_gpt2_cocon = pd.DataFrame()
inpu = []
outp = []
conp = []
outs1 = []
outs2 = []
outs_mp = []

epoch_iterator = tqdm(eval_dataloader, desc="Iteration")
for step, batch in enumerate(epoch_iterator):
    inputs, content, lm_labels = batch
    input_ids = inputs.to(device)
    lm_labels = lm_labels.to(device)
    content = content.to(device)
    for i in range(len(input_ids)):
        #print(tokenizer.decode(input_ids[i]))
        b = input_ids[i]==start_token_id
        indices = b.nonzero()[0][0]
        input_ = input_ids[i][:indices + 1]
        
        b = input_ids[i]==end_token_id
        indices_end = b.nonzero()[0][0]
        
        target = tokenizer.decode(input_ids[i][indices + 1:indices_end])
        #print(tokenizer.decode(input_))
        input_ = tokenizer.decode(input_)
        
        b = content[i]==pad_token_id
        indices = b.nonzero()[0][0]
        content_ = content[i][:indices]
        
        content_ = tokenizer.decode(content_)
        #inpu.append(input_)
        #outp.append(target)
        inpu.append(input_)
        outp.append(target)
        conp.append(content_)
        op=preditction_with_beam_search(input_,content_,2, 10)
        outs1.append(op[0])
        outs2.append(op[1])
        op = generate_gpt(input_,content_,gen_len=10)
        outs_mp.append(op[1][0])
    break
    
df_gpt2_cocon['input'] = inpu
df_gpt2_cocon['gold'] = outp
df_gpt2_cocon['content'] = conp
df_gpt2_cocon['pred1'] = outs1
df_gpt2_cocon['pred2'] = outs2
df_gpt2_cocon['pred_mp'] = outs_mp
    
df_gpt2_cocon.head(1)

Iteration:   0%|                                                                              | 0/2626 [01:20<?, ?it/s]


,input,gold,content,pred1,pred2,pred_mp
0,<POS> <CON_START> i recommend checking this pl...,i highly recommend checking this place out.,highly,.always actors fog fog. fog. fog.,always proposal great actors. fogalways actor...,small definitely. fantastic.always.they recom...


In [49]:
df_gpt2_cocon.to_csv('yelp_models/cocon_gpt_preds.csv', index=False)
df_gpt2_cocon.head()

,input,gold,content,pred1,pred2,pred_mp,plain_pred1,plain_pred2,plain_pred_mp,both_3l_pred1,both_3l_pred2,both_3l_pred_mp
0,<POS> <CON_START> i recommend checking this pl...,i highly recommend checking this place out.,highly,.always actors fog fog. fog. fog.,always proposal great actors. fogalways actor...,small definitely. fantastic.always.they recom...,highly highly checking place. highly highly t...,recommenditageitageitageitageitageitageitagei...,highly highly checking place. highly highly t...,recommend this out.itage actorsigaitage love ...,definitely brushelled highly. highly recommen...,recommend this out you. main mindiquette love...
1,"<NEG> <CON_START> not only is there pizza, but...","not only is there pizza bad, but their custome...",bad horrible,.ues.always actors fog. actors. actors,alwaysalways actorsgreat. great actorsalways ...,.again.wrong always too rude always.att,only is pizza but customer is. Kurd not is,"even there qualified, wantingetooth wanting w...","only is pizza, their service horrible horribl...",also there'); is pizza but customer is..,only is pizza mit decrease collective decreas...,also there is pizza but customer isable <END>...
2,<POS> <CON_START> cool tram that has views goi...,cool tram that has great views going up or dow...,great,always actors actors actors actors actors Kur...,love great always always fog always actors Ku...,owned awesome.keep no months.cious best love,tram has views up down down theitt extended e...,cool always great Mayor MayorGood Mayor pesti...,ixed tram has going or of psburgh skyline <END>,great tram has views up down down the of p,cool Mayor Mayor MayorGood or hotel down pestial,cool that views going or of psburgh skyline the
3,<POS> <CON_START> she was! <START>,she was fantastic!,fantastic,always!always! adulthood always actors adulth...,love. travel.always removing. travel..,spot! delicious always great <END> excellent...,was!she amazing Kurd!she amazing Kurd Kurd,she! Kurd scarthink Quebecthink awesome fogthink,was!she phenomenal <END> was! <END> was!,was!she was! gene! Kurd was!,she. she scar. Kurd actors gene!doors,you was!.. <PAD> <PAD> <PAD> <PAD> <PAD>
4,<NEG> <CON_START> however the food - oh the fo...,however the food - oh the food : ( - i was dis...,disappointed,always. actors actors great. actors actors.al...,. Portlandalways.. Portlandalways.! always,unfortunately.best.worst.always. awful.,ever food oh food ( - i disappointed disappoin...,ever thearia the bad i - scar forecast disapp...,ever food oh food food disgusting ( i sgy,the - the : - food i i WAR.,love food Montgomery Montgomery i tempor temp...,them- food oh nasty food oh terrible!!
